In [52]:
# Load packages
import tensorflow.keras
import pandas as pd
import numpy as np
import math
import glob
import os

In [4]:
# Compile the data

# Lie = 0 and Truth = 1

initialCSV = pd.concat(map(pd.read_csv, glob.glob(os.path.join('ICA', '*.csv'))), ignore_index = True)
    
initialCSV

,EEG.AF3,EEG.T7,EEG.Pz,EEG.T8,EEG.AF4,Truth
0,5.57764,-18.68978,0.12047,-11.03442,13.27478,1
1,12.78836,-43.83118,0.75530,-25.16123,31.04901,1
2,12.26007,-44.01917,1.04022,-24.15714,30.72010,1
3,12.64842,-46.38778,0.76016,-25.72193,32.60343,1
4,15.47116,-55.28282,0.98514,-31.43475,39.30939,1
...,...,...,...,...,...,...
518395,1.28386,-0.26978,-0.13770,-0.49923,-0.40874,0
518396,1.02784,-1.15967,0.04935,-0.17012,-0.45919,0
518397,0.76818,-0.51442,0.05882,0.40821,0.45476,0
518398,1.13579,0.75766,-0.66258,0.18601,1.03034,0


In [88]:
# Partition the data into 80% training data and 20% testing data

num_rows = len(initialCSV.index)
first80p = math.floor(0.8 * num_rows)
last20p = num_rows - first80p

x_train = initialCSV.head(first80p).drop(columns = ['Truth'])
y_train = initialCSV.head(first80p)[['Truth']]

x_test = initialCSV.tail(last20p).drop(columns = ['Truth'])
y_test = initialCSV.tail(last20p)[['Truth']]

x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

print('Shape of x_train:\t' + str(x_train.shape))
print('Shape of y_train:\t' + str(y_train.shape))
print('Shape of x_test:\t' + str(x_test.shape))
print('Shape of y_test:\t' + str(y_test.shape))
print('Number of classes:\t' + str(np.max(y_train) - np.min(y_train) + 1))

Shape of x_train:	(414720, 1, 5)
Shape of y_train:	(414720, 1)
Shape of x_test:	(103680, 1, 5)
Shape of y_test:	(103680, 1)
Number of classes:	2


In [89]:
# One-Hot Encoding
# Transforms a scalar label to a k-dimensional vector
# Lie   = 0 = [ 1 , 0 ]
# Truth = 1 = [ 0 , 1 ]

def to_one_hot(y, num_class = 2):
    
    results = np.zeros((len(y), num_class))
    for i, label in enumerate(y): results[i, label] = 1.
    return results

y_train_vec = to_one_hot(y_train)
y_test_vec = to_one_hot(y_test)

print('Shape of y_train_vec:\t' + str(y_train_vec.shape))
print('Shape of y_test_vec:\t' + str(y_test_vec.shape))

Shape of y_train_vec:	(414720, 2)
Shape of y_test_vec:	(103680, 2)


In [90]:
# Randomly parition the training set into validation and non-validation sets

train_rows = len(y_train_vec)
train_80p = math.floor(0.8 * train_rows)

rand_indices = np.random.permutation(train_rows)
train_indices = rand_indices[0: train_80p]
valid_indices = rand_indices[train_80p: train_rows]

x_trn = x_train[train_indices, :]
y_trn = y_train_vec[train_indices, :]

x_val = x_train[valid_indices, :]
y_val = y_train_vec[valid_indices, :]

print('Shape of x_trn:\t\t' + str(x_trn.shape))
print('Shape of y_trn:\t\t' + str(y_trn.shape))
print('Shape of x_val:\t\t' + str(x_val.shape))
print('Shape of y_val:\t\t' + str(y_val.shape))

Shape of x_trn:		(331776, 1, 5)
Shape of y_trn:		(331776, 2)
Shape of x_val:		(82944, 1, 5)
Shape of y_val:		(82944, 2)


In [91]:
# Build the model

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = models.Sequential()

# Convolutional layers
model.add(layers.Conv1D(256, 1, activation = 'relu', input_shape = (1, 5)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding = 'same'))
model.add(layers.Dropout(0.25))
model.add(layers.Conv1D(128, 1, activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding = 'same'))
model.add(layers.Dropout(0.25))
model.add(layers.Conv1D(64, 1, activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding = 'same'))
model.add(layers.Dropout(0.25))

# Fully-connected layers
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(2, activation = 'softmax'))

model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_32 (Conv1D)          (None, 1, 256)            1536      
                                                                 
 batch_normalization_28 (Bat  (None, 1, 256)           1024      
 chNormalization)                                                
                                                                 
 max_pooling1d_29 (MaxPoolin  (None, 1, 256)           0         
 g1D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 1, 256)            0         
                                                                 
 conv1d_33 (Conv1D)          (None, 1, 128)            32896     
                                                                 
 batch_normalization_29 (Bat  (None, 1, 128)         

In [92]:
# Define model optimizer and loss function

model.compile(
    optimizers.Adam(learning_rate = 0.001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [96]:
# Train the model and store parameters and loss values

history = model.fit(x_trn, y_trn, batch_size = 128, epochs = 50, validation_data = (x_val, y_val))
model.save('seq18.h5')

Epoch 1/50
2592/2592 [==============================] - 55s 21ms/step - loss: 0.6809 - accuracy: 0.5467 - val_loss: 0.6806 - val_accuracy: 0.5514
Epoch 2/50
2592/2592 [==============================] - 56s 21ms/step - loss: 0.6808 - accuracy: 0.5477 - val_loss: 0.6822 - val_accuracy: 0.5496
Epoch 3/50
2592/2592 [==============================] - 55s 21ms/step - loss: 0.6806 - accuracy: 0.5471 - val_loss: 0.6810 - val_accuracy: 0.5510
Epoch 4/50
2592/2592 [==============================] - 55s 21ms/step - loss: 0.6804 - accuracy: 0.5488 - val_loss: 0.6802 - val_accuracy: 0.5518
Epoch 5/50
2592/2592 [==============================] - 56s 21ms/step - loss: 0.6802 - accuracy: 0.5488 - val_loss: 0.6820 - val_accuracy: 0.5526
Epoch 6/50
2592/2592 [==============================] - 57s 22ms/step - loss: 0.6800 - accuracy: 0.5490 - val_loss: 0.6802 - val_accuracy: 0.5527
Epoch 7/50
2592/2592 [==============================] - 55s 21ms/step - loss: 0.6796 - accuracy: 0.5489 - val_loss: 0.6810 -

In [98]:
# Evaluate testing accuracy on the testing dataset 

loss_and_acc = model.evaluate(x_test, y_test_vec)

3240/3240 [==============================] - 6s 2ms/step - loss: 0.7097 - accuracy: 0.4840
